<a href="https://colab.research.google.com/github/111Vidya/PySpark-Fundamentals/blob/main/Apr_24_Linear_Regression_Code_Along.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression Code Along

This notebook is the reference for the video lecture on the Linear Regression Code Along. Basically what we do here is examine a dataset with Ecommerce Customer Data for a company's website and mobile app. Then we want to see if we can build a regression model that will predict the customer's yearly spend on the company's product.

First thing to do is start a Spark Session

In [ ]:
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=cd994fea182db8b131b23bc6929e9513e944911ae9d72c2ec99543ed106d6160
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.ml.regression import LinearRegression # pyspark Linear regression

In [ ]:
# Use Spark to read in the Ecommerce Customers csv file.
data = spark.read.csv("Ecommerce_Customers.csv",inferSchema=True,header=True)

In [ ]:
# Print the Schema of the DataFrame
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [ ]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [ ]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [ ]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


## Setting Up DataFrame for Machine Learning

In [ ]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler # Vector transformation . IV should be clubbed and TV required for ML in pyspark

In [ ]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [ ]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App",
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [ ]:
output = assembler.transform(data)
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [ ]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [ ]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [ ]:
# IV after vaector assembler and TV is yrly amt spent
final_data = output.select("features",'Yearly Amount Spent')

In [ ]:
train_data,test_data = final_data.randomSplit([0.7,0.3]) # split data

In [ ]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                355|
|   mean| 504.23892801074527|
| stddev|   78.8785422476846|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [ ]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                145|
|   mean|  487.2565495550398|
| stddev|  79.35983901971116|
|    min|  275.9184206503857|
|    max|  700.9170916173961|
+-------+-------------------+



In [ ]:
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [ ]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

In [ ]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.66870146747894,38.682045903029994,0.6884950264890846,61.223803132350945] Intercept: -1057.2784030299447


In [ ]:
test_results = lrModel.evaluate(test_data) # test data run model

In [ ]:
# Interesting results....errors
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -12.09219764308034|
| 10.738852024723997|
|  -3.79308267173991|
|  7.285152506355871|
|-5.7532722195606425|
|-12.732177079629707|
| 3.0196855658906543|
| -7.763110884532011|
| -5.638247771025874|
|  -9.66040356606635|
|  6.264006434051623|
|-2.3227214077016356|
|  3.142972506880426|
| -4.837822966153283|
|  -17.7915974204887|
|  8.445052108533957|
| -17.08684037316567|
| 12.336756053590875|
| -2.109338441926127|
|-13.948829449606365|
+-------------------+
only showing top 20 rows



In [ ]:
unlabeled_data = test_data.select('features')

In [ ]:
predictions = lrModel.transform(unlabeled_data)

In [ ]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...|332.02106744627395|
|[30.7377203726281...| 451.0418901715059|
|[30.8794843441274...| 493.9996826565946|
|[30.9716756438877...|487.35345725053685|
|[31.0613251567161...|493.30873027746225|
|[31.0662181616375...|461.66547028730406|
|[31.3091926408918...|  429.701032274043|
|[31.4474464941278...|  426.365852979756|
|[31.5171218025062...| 281.5566684214116|
|[31.5261978982398...|418.75492975840416|
|[31.6548096756927...| 468.9994172934969|
|[31.7216523605090...| 350.0996480395743|
|[31.7366356860502...| 493.7904737486515|
|[31.7656188210424...| 501.3919046017604|
|[31.8164283341993...| 518.9140889241451|
|[31.8512531286083...|464.54719455826444|
|[31.9048571310136...| 491.0366977959818|
|[31.9096268275227...| 551.1092796196483|
|[31.9120759292006...|389.64405474763385|
|[31.9365486184489...|441.14821434493456|
+--------------------+------------

In [ ]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 10.20472188323208
MSE: 104.13634871411571


Excellent results! Let's see how you handle some more realistically modeled data in the Consulting Project!